<a href="https://colab.research.google.com/github/umddb/cmsc424-fall2020/blob/master/UMD_Hackathon21_QASystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#BLOCK 1
#Creates a folder for our Project in Google Colab
!mkdir umd_hackathon2021
%cd umd_hackathon2021

/content/umd_hackathon2021


In [ ]:
#BLOCK 2 
#Makefile script downloads training, testing and development datasets from servers of Amazon Web Services
%%writefile /content/umd_hackathon2021/Makefile
qanta.train.json qanta.test.json qanta.dev.json:
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.train.2018.04.18.json
	mv qanta.train.2018.04.18.json qanta.train.json
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.dev.2018.04.18.json
	mv qanta.dev.2018.04.18.json qanta.dev.json
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.test.2018.04.18.json
	mv qanta.test.2018.04.18.json qanta.test.json
qanta.train.evidence.json qanta.dev.evidence.json qanta.test.evidence.json:
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/rc/evidence_docs_train.json
	mv evidence_docs_train.json qanta.train.evidence.json
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/rc/evidence_docs_dev.json
	mv evidence_docs_dev.json qanta.dev.evidence.json
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/rc/evidence_docs_test.json
	mv evidence_docs_test.json qanta.test.evidence.json
qanta.train.evidence.text.json qanta.dev.evidence.text.json qanta.test.evidence.sent.text.json:
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/rc/evidence_docs_train_with_sent_text.json
	mv evidence_docs_train_with_sent_text.json qanta.train.evidence.text.json
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/rc/evidence_docs_dev_with_sent_text.json
	mv evidence_docs_dev_with_sent_text.json qanta.dev.evidence.text.json
	wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/rc/evidence_docs_test_with_sent_text.json
	mv evidence_docs_test_with_sent_text.json qanta.test.evidence.text.json


Writing /content/umd_hackathon2021/Makefile


In [ ]:
#BLOCK 3
#Downloads the relevant Datasets
!make qanta.train.json

wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.train.2018.04.18.json
--2021-04-10 21:05:14--  https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.train.2018.04.18.json
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.205.120
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.205.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142113194 (136M) [application/json]
Saving to: ‘qanta.train.2018.04.18.json’

qanta.train.2018.04 100%[===================>] 135.53M  36.0MB/s    in 4.3s    

2021-04-10 21:05:19 (31.6 MB/s) - ‘qanta.train.2018.04.18.json’ saved [142113194/142113194]

mv qanta.train.2018.04.18.json qanta.train.json
wget https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.dev.2018.04.18.json
--2021-04-10 21:05:19--  https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.dev.2018.04.1

In [ ]:
#BLOCK 4
#K-NN Classification code to classify answers from questions
%%writefile /content/umd_hackathon2021/knn_hack.py


import argparse
import os
import json
from collections import Counter, defaultdict
from typing import Sequence, Dict

import numpy
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

class Knearest:
    """
    kNN classifier
    """

    def __init__(self, x, y, k):
        """
        Creates a kNN instance
        :param x: Training data input
        :param y: Training data output
        :param k: The number of nearest points to consider in classification
        """

        # You may modify this code, but you shouldn't need to

        self.x = x
        self.y = y
        self.k = k

    def majority(self, item_indices: Sequence[int]) -> str:
        """Given the indices of training examples, return the majority label.
        If there's a tie, return the one that is lexicographically
        first (as determined by python sorted function).
        :param item_indices: The indices of the k nearest neighbors
        (helpfully, this is what's returned by the kneighbors
        function.
        """
        assert len(item_indices) == self.k, "Did not get k inputs"

        # Finish this function to return the most common y value for
        # these indices

        # Gutama and Chiebuka's code block

        freq = Counter(self.y[index] for index in item_indices)
        max_count = freq.most_common(1)[0][1]
        return sorted(i for i in freq if freq[i]  == max_count)[0]

        return None

    def classify(self, example: numpy.ndarray) -> str:
        """
        Given an example, classify the example.
        :param example: A representation of an example in the same
        format as training data
        """
        # Finish this function to find the k closest points, query the
        # majority function, and return the value.
        # Gutama and Chiebuka's code block

        possible_similarities=self.x.dot(example.T).T # [1,2,3,4,4] or [[];[];[]]

        possible_similarities = (-possible_similarities).toarray()
       
        ind = numpy.argsort(possible_similarities, axis=1)[:, 0:self.k]
        
        return self.majority(ind[0])

        return ""

    def confusion_matrix(self, test_x: Sequence[str], test_y: Sequence[str]) -> Dict[str, Dict[str, int]]:
        """
        Given a matrix of test examples and labels, compute the confusion
        matrixfor the current classifier.  Should return a dictionary of
        dictionaries where d[ii][jj] is the number of times an example
        with true label ii was labeled as jj.
        :param test_x: Test data representation
        :param test_y: Test data answers
        """

        # Finish this function to build a dictionary with the
        # mislabeled examples.  You'll need to call the classify
        # function for each example.

        d = defaultdict(dict)

        # David and Nimay's code block 

        index = 0
        
        for tx, ty in zip(test_x, test_y):
          output_label = self.classify(tx)
          d[ty][output_label] = d[ty].get(output_label, 0) + 1
          index += 1
          if (index % 100 == 0):
            print("%i/%i Confusion Matrix" % (index, test_x.shape[0]))
         
        return d

    @staticmethod
    def accuracy(confusion_matrix: Dict[str, Dict[str, int]]) -> float:
        """Given a confusion matrix, compute the accuracy of the underlying
        classifier.
        """

        # Hint: this should give you clues as to how the confusion
        # matrix should be structured.

        total = 0 
        correct = 0
        for ii in confusion_matrix:
            total += sum(confusion_matrix[ii].values())
            correct += confusion_matrix[ii].get(ii, 0)

        return float(correct) / float(total)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='KNN classifier options')
    parser.add_argument("--root_dir", help="QB Dataset for training",
                        type=str, default='../',
                        required=False)
    parser.add_argument("--train_dataset", help="QB Dataset for training",
                        type=str, default='/content/umd_hackathon2021/qanta.train.json',
                        required=False)
    parser.add_argument("--test_dataset", help="QB Dataset for test",
                        type=str, default='/content/umd_hackathon2021/qanta.dev.json',
                        required=False)
    parser.add_argument("--min_df", help="How many documents must a word appear in to be feature",
                        type=int, default=2)
    parser.add_argument("--max_df", help="How many docs can words appear in and still be feature",
                        type=float, default=0.9)
    parser.add_argument("--limit", help="Number of training documents",
                        type=int, default=-1, required=False)
    parser.add_argument("--max_ngram", help="Max ngram length", type=int, default=3)
    parser.add_argument('--k', type=int, default=3,
                        help="Number of nearest points to use")
    args = parser.parse_args()

    # You should not have to modify any of this code
    with open(os.path.join(args.root_dir, args.train_dataset)) as infile:
        data = json.load(infile)["questions"]
        if args.limit > 0:
            data = data[:args.limit]
    vectorizer = TfidfVectorizer(ngram_range=(1, args.max_ngram), min_df=args.min_df, max_df=args.max_df).fit(x["text"] for x in data)
    train_x = vectorizer.transform(x["text"] for x in data)
    train_y = list(x["page"] for x in data)

    print(type(train_x))

    knn = Knearest(train_x, train_y, args.k)
    print("Done loading data")

    with open(os.path.join(args.root_dir, args.test_dataset)) as infile:
        test = json.load(infile)["questions"][:100]

    test_x = vectorizer.transform(x["text"] for x in test)
    test_y = list(x["page"] for x in test)
    answers = [x[0] for x in Counter(test_y).most_common(5)]

    confusion = knn.confusion_matrix(test_x, test_y)
    guesses = set()
    for ii in answers:
        for jj in confusion[ii]:
            guesses.add(jj)

    print("\t" + "\t".join(str(x) for x in answers))
    print("".join(["-"] * 90))
    for ii in guesses:
        print("%30s:\t" % ii + "\t".join(str(confusion[x].get(ii, 0))
                                       for x in answers))
    print("Accuracy: %f" % knn.accuracy(confusion))

Writing /content/umd_hackathon2021/knn_hack.py


In [ ]:
#BLOCK 5
#Test Code to see if the K-NN Classification code works as expected
#Note: Test codes are always useful for debugging purposes
%%writefile /content/umd_hackathon2021/test_hack.py

import unittest

from numpy import array

from knn_hack import *

class TestKnn(unittest.TestCase):
    def setUp(self):
        self.x = array([[2, 0], [4, 1], [6, 0], [1, 4], [2, 4], [2, 5], [4, 4],
                        [0, 2], [3, 2], [4, 2], [5, 2], [7, 3], [5, 5]])
        self.y = array([+1, +1, +1, +1, +1, +1, +1, -1, -1, -1, -1, -1, -1])
        self.knn = {}
        for ii in [1, 2, 3]:
            self.knn[ii] = Knearest(self.x, self.y, ii)

        self.queries = [array(x).reshape(1, -1) for x in
                        [[1, 5], [0, 3], [6, 1], [6, 4]]]
        self.test_y = [1, -1, 1, -1]

    def test1(self):
        self.assertEqual(self.knn[1].classify(self.queries[0]), 1)
        self.assertEqual(self.knn[1].classify(self.queries[1]), -1)
        self.assertEqual(self.knn[1].classify(self.queries[2]), 1)
        self.assertEqual(self.knn[1].classify(self.queries[3]), -1)
        self.assertEqual(self.knn[1].accuracy(self.knn[1].confusion_matrix(self.queries, self.test_y)), 1.0)

    def test2(self):
        self.assertEqual(self.knn[2].classify(self.queries[0]), 1)
        self.assertEqual(self.knn[2].classify(self.queries[1]), -1)
        self.assertEqual(self.knn[2].classify(self.queries[2]), 1)
        self.assertEqual(self.knn[2].classify(self.queries[3]), -1)
        self.assertEqual(self.knn[2].accuracy(self.knn[2].confusion_matrix(self.queries, self.test_y)), 1.0)


    def test3(self):
        self.assertEqual(self.knn[3].classify(self.queries[0]), 1)
        self.assertEqual(self.knn[3].classify(self.queries[1]), 1)
        self.assertEqual(self.knn[3].classify(self.queries[2]), 1)
        self.assertEqual(self.knn[3].classify(self.queries[3]), -1)
        self.assertEqual(self.knn[3].accuracy(self.knn[3].confusion_matrix(self.queries, self.test_y)), 0.75)

if __name__ == '__main__':
    unittest.main()

In [ ]:
#BLOCK 6
#Executes the test code
!python test_hack.py

In [ ]:
#BLOCK 7
#Executes the K-NN Classification algorithm
!python knn_hack.py

<class 'scipy.sparse.csr.csr_matrix'>
Done loading data
100/100 Confusion Matrix
	Texas_annexation	Mark_Antony	Martin_Scorsese	Spin_(physics)	Operation_Condor
------------------------------------------------------------------------------------------
              Texas_annexation:	1	0	0	0	0
                Spin_(physics):	0	0	0	1	0
                   Mark_Antony:	0	1	0	0	0
                     Dirty_War:	0	0	0	0	1
               Martin_Scorsese:	0	0	1	0	0
Accuracy: 0.480000


In [ ]:
#Block 8 FINAL CODE (Buzz for answers above a threshold value and generate a csv file adapting knn_hack.py in Block 4)
#K-NN Classification code to classify answers from questions
#Please note that the automatically generated csv files had to be minimally cleaned for the purpose of the 24 hour Hackathon
#Further research is needed to identify a good threshold value
%%writefile /content/umd_hackathon2021/knn_hack_thresh_buzzer.py


import argparse
import os
import json
from collections import Counter, defaultdict
from typing import Sequence, Dict

import numpy
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import csv

THRESHOLD = 0.21

class Knearest:
    """
    kNN classifier
    """

    def __init__(self, x, y, k):
        """
        Creates a kNN instance
        :param x: Training data input
        :param y: Training data output
        :param k: The number of nearest points to consider in classification
        """

        # You may modify this code, but you shouldn't need to

        self.x = x
        self.y = y
        self.k = k

    def majority(self, item_indices: Sequence[int]) -> str:
        """Given the indices of training examples, return the majority label.
        If there's a tie, return the one that is lexicographically
        first (as determined by python sorted function).
        :param item_indices: The indices of the k nearest neighbors
        (helpfully, this is what's returned by the kneighbors
        function.
        """
        assert len(item_indices) == self.k, "Did not get k inputs"

        # Finish this function to return the most common y value for
        # these indices

        # Gutama and Chiebuka's code block
        
        freq = Counter(self.y[index] for index in item_indices)
        max_count = freq.most_common(1)[0][1]
        output_label = sorted(i for i in freq if freq[i]  == max_count)[0]
        # Hardcoding for now since k = 3
        if self.y[item_indices[0]] == output_label:
           index_output_label = item_indices[0]
        elif self.y[item_indices[1]] == output_label:
           index_output_label = item_indices[1]
        elif self.y[item_indices[2]] == output_label:
           index_output_label = item_indices[2]
        return output_label, index_output_label
        return None

    def classify(self, example: numpy.ndarray) -> str:
        """
        Given an example, classify the example.
        :param example: A representation of an example in the same
        format as training data
        """
        # Finish this function to find the k closest points, query the
        # majority function, and return the value.
        # Gutama and Chiebuka's code block
        possible_similarities=self.x.dot(example.T).T # [1,2,3,4,4] or [[];[];[]]

        possible_similarities = (-possible_similarities).toarray()
       
        ind = numpy.argsort(possible_similarities, axis=1)[:, 0:self.k]
        
        output_label, index_output_label = self.majority(ind[0])

        confidence_value = self.x[index_output_label].dot(example.T).T
        return output_label, confidence_value

    def confusion_matrix(self, test_x, qid_sentid_test_x, test_y: Sequence[str]) -> Dict[str, Dict[str, int]]:
        """
        Given a matrix of test examples and labels, compute the confusion
        matrixfor the current classifier.  Should return a dictionary of
        dictionaries where d[ii][jj] is the number of times an example
        with true label ii was labeled as jj.
        :param test_x: Test data representation
        :param test_y: Test data answers
        """

        # Finish this function to build a dictionary with the
        # mislabeled examples.  You'll need to call the classify
        # function for each example.

        d = defaultdict(dict)

        # David and Nimay's initial code block Updated by Gutama and Chiebuka for final buzzer

        index = 0
        # values to be stored in the buzz csv file
        buzz_row_headers = ['question',	'sentence', 'word',	'page',	'evidence', 'final',	'weight'] 
        # values to be stored in the final csv file
        final_row_headers = ['question','answer']
        
        buzz_file = open('UMD_Hackathon.buzz.csv', 'w', newline='')          
        buzzwriter = csv.writer(buzz_file)
        buzzwriter.writerow(buzz_row_headers)

        final_file = open('UMD_Hackathon.final.csv', 'w', newline='')
        finalwriter = csv.writer(final_file)
        finalwriter.writerow(final_row_headers)

        prev_qid = -1
        final_found_flag = 0
        past_confidence_score = 0
        past_output_label = ''
        past_id_buzz0 = 0
        past_output_label_buzz0 = ''
        final_buzz = 1
        buzz_rows = []
        final_rows = []
        len_all_sentences_all_questions = len(qid_sentid_test_x)
        
        i = 0 
        print("Data type of dict after call ",type(test_x))
        for tx, ty in zip(test_x, test_y):
          if (prev_qid == -1):
              if (i<len_all_sentences_all_questions):
                  qid = qid_sentid_test_x[i][0]
                  sentid = qid_sentid_test_x[i][1]
              i+=1 
          if (prev_qid != qid):
              prev_qid = qid
              final_found_flag = 0
              if final_buzz == 0:
                 final_rows.append([past_id_buzz0, past_output_label_buzz0])


          if (final_found_flag == 0):
              output_label, confidence_score  = self.classify(tx)
              
              if (prev_qid != -1 and i<len_all_sentences_all_questions):
                  qid = qid_sentid_test_x[i][0]
                  sentid = qid_sentid_test_x[i][1]
              i+=1
              d[ty][output_label] = d[ty].get(output_label, 0) + 1
              index += 1
              if (index % 100 == 0):
                  print("%i/%i Confusion Matrix" % (index, test_x.shape[0]))
          else:
              if (i<len_all_sentences_all_questions):
                  qid = qid_sentid_test_x[i][0]
                  sentid = qid_sentid_test_x[i][1]
              i+=1
              buzz_rows.append([qid, sentid,0,past_output_label,'',1,past_confidence_score[0][0]])
              d[ty][past_output_label] = d[ty].get(past_output_label, 0) + 1
              index += 1
              if (index % 100 == 0):
                  print("%i/%i Confusion Matrix" % (index, test_x.shape[0]))
              continue

                                                        
          if (confidence_score[0][0]>=THRESHOLD):
                  final_buzz = 1   
                  buzz_rows.append([qid, sentid,0,output_label,'',final_buzz,confidence_score[0][0]])

                  if (final_found_flag != 1):
                    final_rows.append([qid, output_label])
                  final_found_flag = 1
                  past_confidence_score = confidence_score[0][0]
                  past_output_label = output_label
          else:
                  final_buzz = 0 #TASK 5 Value goes to csv file along with associate confidence_score and sentence_number (which is the iteration number in this current for loop)
                  buzz_rows.append([qid, sentid,0,output_label,'',final_buzz,confidence_score[0][0]])
                  
                  past_id_buzz0 = qid
                  past_output_label_buzz0 = output_label
        

        buzzwriter.writerows(buzz_rows)
        finalwriter.writerows(final_rows)

        return d

    @staticmethod
    def accuracy(confusion_matrix: Dict[str, Dict[str, int]]) -> float:
        """Given a confusion matrix, compute the accuracy of the underlying
        classifier.
        """

        # Hint: this should give you clues as to how the confusion
        # matrix should be structured.

        total = 0 
        correct = 0
        for ii in confusion_matrix:
            total += sum(confusion_matrix[ii].values())
            correct += confusion_matrix[ii].get(ii, 0)
        
        return float(correct) / float(total)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='KNN classifier options')
    parser.add_argument("--root_dir", help="QB Dataset for training",
                        type=str, default='../',
                        required=False)
    parser.add_argument("--train_dataset", help="QB Dataset for training",
                        type=str, default='/content/umd_hackathon2021/qanta.train.json',
                        required=False)
    parser.add_argument("--test_dataset", help="QB Dataset for test",
                        type=str, default='/content/umd_hackathon2021/qanta.dev.json',
                        required=False)
    parser.add_argument("--min_df", help="How many documents must a word appear in to be feature",
                        type=int, default=2)
    parser.add_argument("--max_df", help="How many docs can words appear in and still be feature",
                        type=float, default=0.9)
    parser.add_argument("--limit", help="Number of training documents",
                        type=int, default=-1, required=False)
    parser.add_argument("--max_ngram", help="Max ngram length", type=int, default=3)
    parser.add_argument('--k', type=int, default=3,
                        help="Number of nearest points to use")
    args = parser.parse_args()

    # You should not have to modify any of this code
    with open(os.path.join(args.root_dir, args.train_dataset)) as infile:
        data = json.load(infile)["questions"]
        if args.limit > 0:
            data = data[:args.limit]
    vectorizer = TfidfVectorizer(ngram_range=(1, args.max_ngram), min_df=args.min_df, max_df=args.max_df).fit(x["text"] for x in data)
    train_x = vectorizer.transform(x["text"] for x in data)
    train_y = list(x["page"] for x in data)

    knn = Knearest(train_x, train_y, args.k)
    print("Done loading data")

    dict_test_x = {} 
    test_y = []
    all_questions = []
    with open('UMD_Hackathon.csv', 'r', newline='') as csvfile: 
         reader = csv.DictReader(csvfile)
         prev_qid = -1 
         prev_qid_list = []
         all_testx_vec = []
         for row in reader: 
             if (prev_qid!=row["id"] and prev_qid!=-1):
               prev_qid = row["id"]
               vec_test_x = vectorizer.transform(each_sent_question for each_sent_question in per_question) 
               all_testx_vec.append(vec_test_x)
               per_question = []
               test_y.append(row["answer"])

             if prev_qid == -1:
               test_y.append(row["answer"])
               prev_qid = row["id"]
               #per_question = ''
               per_question = []

             if (prev_qid == row["id"]):
               prev_qid_list.append([row["id"],row["sent"]])
               per_question.append(row["text"])
    
    #For the last question
    vec_test_x = vectorizer.transform(each_sent_question for each_sent_question in per_question) 
    all_testx_vec.append(vec_test_x)

    confusion = knn.confusion_matrix(all_testx_vec, prev_qid_list, test_y)

    #print("Data type of dict b4 call ",type(dict_test_x))
    #confusion = knn.confusion_matrix(dict_test_x, test_y)
    
    answers = [x[0] for x in Counter(test_y).most_common(5)]

    guesses = set()
    for ii in answers:
        for jj in confusion[ii]:
            guesses.add(jj)

    print("\t" + "\t".join(str(x) for x in answers))
    print("".join(["-"] * 90))
    for ii in guesses:
        print("%30s:\t" % ii + "\t".join(str(confusion[x].get(ii, 0))
                                       for x in answers))
    print("Accuracy: %f" % knn.accuracy(confusion))

Overwriting /content/umd_hackathon2021/knn_hack_thresh_buzzer.py


In [ ]:
!python3 knn_hack_thresh_buzzer.py